In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#covert the time string to a number
def time_to_num(t):
    num = int(t[0:2]) * 3600 + int(t[3:5]) * 60 + int(t[6:8])
    return num

In [3]:
#generate the label
def get_label(df, future_step):
    label = ((df['AskPrice1'].shift(-future_step) + df['BidPrice1'].shift(-future_step)) 
    - (df['AskPrice1'] + df['BidPrice1'])) / 2
    return label

In [4]:
raw_data = pd.read_csv('data/data.csv')

In [37]:
raw_data['UpdateTime'] = raw_data['UpdateTime'].map(time_to_num)

In [38]:
raw_data['label'] = get_label(raw_data,10)

In [39]:
#make sure the label and feature are in the same day
data_with_label = raw_data[raw_data['UpdateTime'].shift(-10) >= raw_data['UpdateTime']]

In [40]:
data_with_label.dropna()

,indicator1,indicator2,indicator3,indicator4,indicator5,indicator6,indicator7,indicator8,indicator9,indicator10,...,AskVolume1,BidVolume1,BidVolume2,BidVolume3,BidVolume4,BidVolume5,OpenInterest,UpperLimitPrice,LowerLimitPrice,label
0,1.000090,-0.086505,-1374.480,-2.076120,0.000178,0.999911,0.998495,0.750000,1.038060,0.004270,...,2,1,28,4,62,2,89336,5869,5311,1.0
1,1.000090,-0.086505,-1374.480,0.000000,0.000178,0.999911,0.998506,0.500000,0.000000,0.000000,...,2,1,28,4,62,2,89336,5869,5311,1.0
2,1.000090,-0.086505,-1374.480,0.000000,0.000178,1.000090,0.998516,0.500000,0.000000,0.000712,...,2,1,28,4,62,2,89336,5869,5311,1.0
3,1.000090,0.000000,490.311,1.038060,0.000178,1.000090,0.998703,0.666667,0.346021,0.002134,...,2,2,1,28,7,62,89340,5869,5311,0.0
4,1.000090,0.000000,571.371,0.000000,0.000178,1.000090,0.998712,0.500000,0.000000,0.000000,...,2,2,1,28,7,62,89340,5869,5311,0.0
5,1.000090,1.411290,8503.230,0.000000,0.000178,1.000090,0.998721,0.500000,0.000000,0.000000,...,2,16,17,28,7,62,89340,5869,5311,0.0
6,0.999911,1.008060,6239.110,1.008060,0.000178,1.000000,0.998907,0.500000,0.000000,0.001773,...,6,16,18,28,7,62,89340,5869,5311,0.0
7,1.000090,-0.403226,-1690.320,-2.016130,0.000178,1.000000,0.998739,0.950000,1.814520,0.003549,...,10,6,19,28,7,62,89358,5869,5311,0.0
8,1.000090,-0.302419,-1123.790,0.000000,0.000178,1.000000,0.998749,0.500000,0.000000,0.000000,...,10,7,20,28,7,62,89358,5869,5311,0.0
9,1.000090,-0.504032,-2256.450,-0.403226,0.000178,1.000000,0.998757,1.000000,0.403226,0.000710,...,10,5,20,28,7,62,89362,5869,5311,0.0


In [41]:
tmp = data_with_label['UpdateTime'].shift(-1) - data_with_label['UpdateTime']

In [43]:
end_day = np.where(tmp < 0)[0].tolist()

In [49]:
begin = 0
data_in_day = []
for i in end_day:
    data_in_day.append(data_with_label[begin: i + 1])
    begin = i + 1
data_in_day.append(data_with_label[begin: data_with_label.shape[0]])

In [53]:
trainning_set, testing_set = train_test_split(data_in_day, test_size = 0.2, random_state = 37)

In [55]:
len(trainning_set)

36

In [56]:
len(testing_set)

9